# Predicting Airbnb rental prices

Airbnb는 다른 사람에게 거주 공간을 단기 임대할 수 있도록 중개하는 플랫폼입니다. 2008년에 세워진 Airbnb의 회사 가치는 2016년 300억 달러 상당에서 현재는 전 세계 어떤 호텔 체인보다도 높습니다.

Airbnb에 처음 호스팅하는 사람이 겪는 어려움 중 하나는 적절한 숙박료를 정하는 일입니다.

Airbnb에 등록된 숙소라면 어떤 지역이든 숙박객은 가격, 침실의 개수, 숙소의 종류 등과 같은 기준에 따라 자유롭게 숙소를 고를 수 있습니다. 또 Airbnb에서는 누구나 호스트가 될 수 있기 때문에 호스트가 매기는 숙박료는 시장의 성격에 큰 영향을 받습니다

만약 호스트가 시장 가격 이상으로 숙박료를 비싸게 매기면 숙박객은 다른 숙소를 찾을 것입니다. 반대로 숙박료를 너무 낮게 매기면 충분한 이윤을 얻을 수 없습니다.

이때 가능한 한 가지 전략은:

내 숙소와 비슷한 몇 개의 숙소를 찾은 다음, 이들 숙소의 평균 가격을 숙박료로 정하는 것입니다. k-nearest neighbors라고 불리는 알고리즘을 이용해 이 과정을 자동화해줄 머신러닝 모델을 만들것입니다.

먼저 데이터셋부터 살펴봅시다.

# Airbnb-scraper

In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import math
from pprint import pprint
from urllib.parse import urlparse
from datetime import datetime
import pandas as pd
from selenium.common.exceptions import NoSuchElementException  
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import time


In [3]:

"""
helpers
"""
def isfloat(value):
    """
    isdigit과 비슷한 동작을 수행함. 
    """
    try:
        float(value)
        return True
    except ValueError:
        return False

In [4]:
def extract_numbers(string):
    """
    숙소에 대한 설명 등과 같은 스트링에서 숫자 정보만을 추출.
    """
    float_numbers = [float(s) for s in string[:-1].split() if isfloat(s)]
    default = 1
    
    try:
        default = float_numbers[0]
    except IndexError:
        print('입력 스트링에서 숫자를 찾을 수 없습니다.')
        
    return default

In [5]:
def check_element_exists(driver, selector):
    try:
        driver.find_elements_by_css_selector(selector)
    except NoSuchElementException:
        return False
    return True

In [6]:
def get_id_from_url(url):
    return int(urlparse(url).path.split('/')[-1])

def get_text_from_tag(driver, selector):
    tag = driver.find_element_by_css_selector(selector)
    return tag.text or 'N/A'

In [13]:
# 크롬드라이브 설치
pwd

NameError: name 'pwd' is not defined

In [14]:
"""
application level constants
"""
WAIT_FOR_SECOND = 3

# chrome driver가 존재하는 디렉토리
driverLocation = 'C:/Users/MyCom/jupyter-tutorial/kaggle/Airbnb-price-prediction/data/chromedriver.exe'


In [15]:
"""
crawler
"""
def get_available_hotels(driver, query):
    """
    지정된 쿼리 파라미터에 따라 에어비앤비 숙소 검색을 수행.
    페이지 아래쪽에 있는 페이지네이션 버튼으로부터 최대 페이지를 알아낸 다음,
    각 숙소의 url을 가져와서 저장하고 다음 페이지로 넘어감 (클릭).
    """    
    try:
        url = f'https://www.airbnb.co.kr/s/homes?query={query["query"]}&checkin={query["checkin"]}&checkout={query["checkout"]}'

        driver.get(url)

        available_hotels = []
    
        query_results_summary_tag = 'div[itemprop="itemList"]'
    
        WebDriverWait(driver, WAIT_FOR_SECOND) \
            .until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, query_results_summary_tag)))
        
        no_results_tag = '._76dwae > ._jmmm34f'
        
        if not check_element_exists(driver, no_results_tag):
            return print('검색 결과가 존재하지 않습니다.')
        
        PAGE_NUM = 1
        MAX_PAGENUM = math.inf

        while PAGE_NUM <= MAX_PAGENUM:
            hotel_link_tag = '._1szwzht a'

            if check_element_exists(driver, hotel_link_tag): 
                hotel_links = WebDriverWait(driver, WAIT_FOR_SECOND) \
                    .until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, hotel_link_tag)))

                for link in hotel_links:
                    available_hotels.append( link.get_attribute('href') )

                pagination_group_tag = '._11hau3k li[data-id^="page-"]'

                pagination_buttons = WebDriverWait(driver, WAIT_FOR_SECOND) \
                    .until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, pagination_group_tag)))

                # get page number of last pagination button
                MAX_PAGENUM = int( pagination_buttons[-1].get_attribute('data-id').replace('page-', '') )

                for button in pagination_buttons:
                    button_number = button.get_attribute('data-id')
                
                    if button_number == f'page-{PAGE_NUM}':
                        button.click()
                        break
                    
                PAGE_NUM += 1
                
                time.sleep(3)

        return available_hotels
        
    except TimeoutException:
        print('숙소 검색 결과가 존재하지 않습니다.')

def get_each_hotel(driver, url):
    """
    각 숙소 url을 받아서 아래의 숙소 디테일 페이지로부터 아래의 항목을 수집함.
    
    id: airbnb 숙소 아이디
    accommodates: 인원
    bathrooms: 욕실 수
    beds: 침대 수
    bedrooms: 침실 수
    nightly_price: 가격
    number_of_reviews: 후기 개수
    room_type: 다인실
    house_name: 숙소명
    """
    try:
        driver.get(url)
        
        price_tag = '._doc79r'
        
        WebDriverWait(driver, WAIT_FOR_SECOND) \
                .until(EC.presence_of_element_located((By.CSS_SELECTOR, price_tag)))
        
        driver.execute_script("window.scrollTo(0, 300)")
        
        # airbnb uses diverse set of tags for hotel meta information
        hotel_meta_tags = [
            '._n5lh69r ._fgdupie',
            '._1thk0tsb > ._12i0h32r',
            '._36rlri > ._ncwphzu',
            '._qtix31 > ._1thk0tsb > ._fgdupie'
        ]
        
        meta = []
        
        for tag in hotel_meta_tags:
            meta_elements = driver.find_elements_by_css_selector(tag)
            
            if driver.find_elements_by_css_selector(tag):
                meta = meta_elements
        
        try:
            accomodates = extract_numbers( meta[0].text )
            bedrooms =  extract_numbers( meta[1].text )
            beds =  extract_numbers( meta[2].text )
            bathrooms = extract_numbers( meta[3].text )
        except IndexError:
            return print('숙소의 메타 태그를 찾을 수 없습니다. 다른 css 선택자로 시도해보세요.')
        
        room_type_tag = '._1hh2h7tb > span'
        hotel_name_tag = '._12ei9u44 > ._fecoyn4'
        number_of_reviews_tag = '._492uxj4 > ._fecoyn4'
        nightly_price_tag = '._doc79r'
        
        def convert_currency_to_float(currency_str):
            return currency_str[1:].replace(',', '')
        
        _id = get_id_from_url(url)
        room_type = get_text_from_tag(driver, room_type_tag)
        hotel_name = get_text_from_tag(driver, hotel_name_tag)
        number_of_reviews = extract_numbers(number_of_reviews_tag)
        nightly_price = convert_currency_to_float( get_text_from_tag(driver, nightly_price_tag) )

        data = {
            'id': _id,
            'accomodates': accomodates,
            'bedrooms': bedrooms,
            'beds': beds,
            'bathrooms': bathrooms,
            'room_type': room_type,
            'hotel_name': hotel_name,
            'number_of_reviews': number_of_reviews,
            'nightly_price': nightly_price,
        }

        return data
    except TimeoutException:
        return print('숙소를 찾을 수 없습니다.')

In [17]:
    
def write_excel(data, fileName):
    """
    엑셀 파일을 저장.
    """
    df = pd.DataFrame(data)
    
    writer = pd.ExcelWriter(fileName, engine='xlsxwriter')
    
    df.to_excel(writer, sheet_name='listing')
    
    writer.save()

def main():
    """
    실제로 크롤링을 수행하는 함수.
    """
    print('에어비앤비 크롤러입니다.')
    
    query = input('숙소가 위치한 지역명을 입력해주세요: ')
    checkin = input('체크인 날짜를 2021-10-08와 같은 형식으로 입력해주세요: ')
    checkout = input('체크아웃 날짜를 2021-10-14와 같은 형식으로 입력해주세요: ')
    
    query_params = {
        'query': query,
        'checkin': checkin,
        'checkout': checkout,
    }
    
    print(query_params)
    
    print('브라우저 창이 열릴 때까지 기다려주세요...')
        
    driver = webdriver.Chrome(driverLocation)
    
    available_hotel_urls = get_available_hotels(driver, query=query_params)
    
    if available_hotel_urls:
        data = []

        for url in available_hotel_urls:
            hotel_data = get_each_hotel(driver, url)
            
            if hotel_data:
                data.append(hotel_data)
            
        timestamp = datetime.today().strftime('%Y-%m-%d %H-%M')
        fileName = f'listing-{query}-{timestamp}.xlsx'

        write_excel(data, fileName)

        driver.close()


main()

에어비앤비 크롤러입니다.
숙소가 위치한 지역명을 입력해주세요: 서울 강남구
체크인 날짜를 2021-10-08와 같은 형식으로 입력해주세요: 2021-10-08
체크아웃 날짜를 2021-10-14와 같은 형식으로 입력해주세요: 2021-10-14
{'query': '서울 강남구', 'checkin': '2021-10-08', 'checkout': '2021-10-14'}
브라우저 창이 열릴 때까지 기다려주세요...
숙소 검색 결과가 존재하지 않습니다.
